# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [103]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Exercise').getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [104]:
df = spark.read.csv('walmart_stock.csv', header=True, inferSchema=True)
df.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

#### What are the column names?

In [105]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [106]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [107]:
df.head(5)

[Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

#### Use describe() to learn about the DataFrame.

In [108]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [109]:
from pyspark.sql.functions import format_number

In [110]:
desc = df.describe()

In [111]:
desc.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [112]:
from pyspark.sql.functions import format_number

In [113]:
describe = desc.select(desc['summary'], format_number(desc['open'].cast('float'),2).alias('Open'),
                      format_number(desc['high'].cast('float'),2).alias('High'),
                      format_number(desc['low'].cast('float'),2).alias('Low'),
                      format_number(desc['close'].cast('float'),2).alias('Close'),
                      format_number(desc['volume'].cast('int'),2).alias('Volume')
                      )
describe.show()

+-------+--------+--------+--------+--------+-------------+
|summary|    Open|    High|     Low|   Close|       Volume|
+-------+--------+--------+--------+--------+-------------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.00|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,780.00|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,100.00|
+-------+--------+--------+--------+--------+-------------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [114]:
hv = df.select(df['High']/df['Volume'])
hv.withColumnRenamed('(High / Volume)','HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [115]:
from pyspark.sql.functions import max, to_timestamp
max_val = df.select(max(df['high'])).collect()[0][0]

In [116]:
dfFiltered = df.filter(df['high']==max_val)
dfFiltered.show()

+----------+---------+---------+-----+---------+-------+---------+
|      Date|     Open|     High|  Low|    Close| Volume|Adj Close|
+----------+---------+---------+-----+---------+-------+---------+
|2015-01-13|90.800003|90.970001|88.93|89.309998|8215400|83.825448|
+----------+---------+---------+-----+---------+-------+---------+



In [117]:
dfFiltered.select(to_timestamp(dfFiltered['Date'], 'yyyy-MM-dd')).collect()[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

#### What is the mean of the Close column?

In [118]:
from pyspark.sql.functions import avg
df.select(avg(df['close'])).show()

+-----------------+
|       avg(close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [119]:
from pyspark.sql.functions import max, min
df.select(max(df['volume']), min(df['volume'])).show()

+-----------+-----------+
|max(volume)|min(volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [120]:
from pyspark.sql.functions import count
dfFiltered = df.filter(df['close']<60)
dfFiltered.select(count(dfFiltered['Date'])).collect()[0][0]

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [121]:
from pyspark.sql.functions import count
total_days = df.select(count(df['Date'])).collect()[0][0]

In [122]:
dfFiltered = df.filter(df['high']>80)
high80_days = dfFiltered.select(count(dfFiltered['Date'])).collect()[0][0]

In [123]:
(high80_days*100)/total_days

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [124]:
from pyspark.sql.functions import corr
df.select(corr(df['high'],df['volume'])).show()

+-------------------+
| corr(high, volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [125]:
from pyspark.sql.functions import year

df.select(year(df['Date']).alias('year'), df['high']).groupby('year').agg({'high':'max'}).show()

+----+---------+
|year|max(high)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [126]:
from pyspark.sql.functions import month,avg

df.select(month(df['Date']),'close').groupby('month(Date)').agg({'close':'avg'}).show()

+-----------+-----------------+
|month(Date)|       avg(close)|
+-----------+-----------------+
|         12|72.84792478301885|
|          1|71.44801958415842|
|          6| 72.4953774245283|
|          3|71.77794377570092|
|          5|72.30971688679247|
|          9|72.18411785294116|
|          4|72.97361900952382|
|          8|73.02981855454546|
|          7|74.43971943925233|
|         10|71.57854545454543|
|         11| 72.1110893069307|
|          2|  71.306804443299|
+-----------+-----------------+



# Great Job!